In [14]:
import sys
sys.path.append("/Users/tristan/Build/petsc/lib/petsc/bin")
from pyamg import *
from PetscBinaryIO import PetscBinaryIO
import h5py
import scipy as sp
from scipy.sparse import *

In [16]:
io = PetscBinaryIO()
S = io.readBinaryFile("../ba-tao/ladybug-1723-S.petsc", mattype="scipy.sparse")[0]
S = bsr_matrix(S, blocksize=(9,9))
rhs = io.readBinaryFile("../ba-tao/ladybug-1723-S-rhs.petsc")[0]

In [20]:
sa = smoothed_aggregation_solver(S)
print(sa)
residuals = []
sa.solve(rhs, residuals=residuals, accel="cg", maxiter=800, tol=1e-8)
print("{} iterations, detla: {}".format(len(residuals), residuals[-1]/residuals[0]))

multilevel_solver
Number of Levels:     3
Operator Complexity:  1.000
Grid Complexity:      1.007
Coarse Solver:        'pinv2'
  level   unknowns     nonzeros
    0        15507     19222677 [99.95%]
    1           99         9153 [ 0.05%]
    2            9           81 [ 0.00%]

204 iterations, detla: 9.602713267682027e-09


In [44]:
l = []
with(h5py.File('../sba_multigrid/ladybug-1723-S-smallest-0.h5', 'r')) as f:
    for i in range(0,9):
        l.append(np.array(f['eigenvector'+str(i)]).flatten())
B = np.vstack(l).transpose()
sa = smoothed_aggregation_solver(S, B)
print(sa)
sa.solve(rhs, residuals=residuals, accel="cg", maxiter=800, tol=1e-8)
print("{} iterations, detla: {}".format(len(residuals), residuals[-1]/residuals[0]))

multilevel_solver
Number of Levels:     3
Operator Complexity:  1.000
Grid Complexity:      1.007
Coarse Solver:        'pinv2'
  level   unknowns     nonzeros
    0        15507     19222677 [99.95%]
    1           99         9153 [ 0.05%]
    2            9           81 [ 0.00%]

173 iterations, detla: 9.283557878675238e-09


In [51]:
l = []
with(h5py.File('../sba_multigrid/ladybug-1723-SP-smallest-0.h5', 'r')) as f:
    for i in range(0,9):
        l.append(np.array(f['eigenvector'+str(i)]).flatten())
B = np.vstack(l).transpose()
sa = smoothed_aggregation_solver(S, B, strength="evolution")
print(sa)
sa.solve(rhs, residuals=residuals, accel="cg", maxiter=800, tol=1e-8)
print("{} iterations, detla: {}".format(len(residuals), residuals[-1]/residuals[0]))

multilevel_solver
Number of Levels:     3
Operator Complexity:  1.012
Grid Complexity:      1.034
Coarse Solver:        'pinv2'
  level   unknowns     nonzeros
    0        15507     19222677 [98.82%]
    1          486       227286 [ 1.17%]
    2           36         1296 [ 0.01%]

81 iterations, detla: 9.974450249544595e-09


In [48]:
l = []
with(h5py.File('../sba_multigrid/ladybug-1723-H-smallest-0.h5', 'r')) as f:
    for i in range(0,9):
        l.append(np.array(f['eigenvector'+str(i)]).flatten())
B = np.vstack(l).transpose()[0:1723*9,:]
sa = smoothed_aggregation_solver(S, B)
print(sa)
sa.solve(rhs, residuals=residuals, accel="cg", maxiter=800, tol=1e-8)
print("{} iterations, detla: {}".format(len(residuals), residuals[-1]/residuals[0]))

multilevel_solver
Number of Levels:     3
Operator Complexity:  1.000
Grid Complexity:      1.007
Coarse Solver:        'pinv2'
  level   unknowns     nonzeros
    0        15507     19222677 [99.95%]
    1           99         9153 [ 0.05%]
    2            9           81 [ 0.00%]

194 iterations, detla: 9.682751701104587e-09


In [61]:
print("H")
print(np.array(h5py.File('../sba_multigrid/ladybug-1723-H-smallest-0.h5', 'r')['eigenvalues'])[0:8])
print("S scaled, no diag")
print(np.array(h5py.File('../sba_multigrid/ladybug-1723-SP-smallest-0.h5', 'r')['eigenvalues'])[0:100])
print("S scaled, diag")
print(np.array(h5py.File('../sba_multigrid/ladybug-1723-S-smallest-0.h5', 'r')['eigenvalues'])[0:50])

H
[1.03159479e-21 1.51797135e-12 1.20235883e-08 8.86130819e-07
 2.22837840e-06 5.16155510e-05 1.22928158e-04 2.77303005e-04]
S scaled, no diag
[ 2.57217680e-16 -2.80222118e-16 -3.69134662e-16  4.36126988e-16
  5.33349175e-16  6.04716884e-16 -6.08063029e-16  6.57249303e-16
 -7.58732858e-16  7.71062486e-16  8.80512744e-16 -1.03496573e-15
  1.11687088e-15 -1.14415027e-15 -1.24673547e-15 -1.59052517e-15
  1.70953076e-15  1.76521201e-15  1.81237801e-15  2.27122638e-15
  3.21032609e-15  4.35957859e-15  6.75064800e-15  1.08718713e-14
 -1.25755024e-14  1.43624713e-14  3.06346509e-14  3.78038499e-14
  8.81852607e-14 -1.42892741e-13  2.77249056e-13  3.93208664e-13
 -4.42129955e-13  5.68481059e-13  6.39427937e-13  8.13514992e-13
 -8.92656484e-13  1.20759027e-12  2.60367378e-12  2.92454709e-12
  4.30094186e-12  4.80763120e-12  9.97226460e-12 -1.21240401e-11
  1.69928736e-11  1.72876282e-11  1.90760546e-11  2.06484817e-11
  4.86819901e-11  5.33235678e-11  6.17190973e-11  7.38170252e-11
  9.98609522